# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Aprendizaje no supervisado

### Trabajo práctico entregable - Grupo 22 - FIFA female players 2023

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

Inicializamos el entorno

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

Cargamos el conjunto de datos y vemos su contenido.

In [ ]:
path = 'female_players.csv'
fifa = pd.read_csv(path)

---
# Pretratamiento

## Tamaño del dataset

In [ ]:
print(f'La base de datos original consta de {fifa.shape[0]} registros y {fifa.shape[1]} variables.')

print(f"Tiene datos desde la versión {fifa['fifa_version'].min()} hasta la versión {fifa['fifa_version'].max()}")

print('\n\nComo nos interesan sólo las jugadoras del 2023, filtramos el dataset para quedarnos con este subconjunto.')

fifa23 = fifa[fifa['fifa_version'] == 23].copy()

print(f'La base de datos 2023 consta de {fifa23.shape[0]} registros y {fifa23.shape[1]} variables.')

fifa23

## Reducción de variables

Vamos a eliminar aquellas columnas que creemos que no serán relevantes para nuestro objetivo.

In [ ]:
fifa23.drop(inplace=True, columns=['player_id', 'player_url', 'fifa_version', 
            'fifa_update', 'fifa_update_date', 'long_name', 'value_eur', 
            'wage_eur', 'dob', 'league_id', 'league_name', 'league_level', 
            'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 
            'club_loaned_from', 'club_joined_date', 
            'club_contract_valid_until_year', 'nationality_id', 
            'nation_team_id', 'nation_position', 'nation_jersey_number', 
            'weight_kg', 'skill_moves', 'international_reputation', 'body_type',
            'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 
            'player_face_url'])

In [ ]:
print(f'Ahora tenemos {fifa23.shape[1]} variables ({110-fifa23.shape[1]} menos).')

fifa23

## Datos faltantes

Tenemos valores faltantes en `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic` y `goalkeeping_speed`. Entre las primeras 6 variables el mapa de calor de missingno nos arroja un valor de +1.00, lo cual indica que existe una probabilidad del 100% de que cuando falte un valor en la variable que está en la fila, va a faltar en la variable que está en la columna. Por ejemplo, cuando falta en `shooting`, también falta en `pace` y así. Para el caso de la variable `goalkeeping_speed` pasa lo opuesto, ya que tiene un valor de -1.00: siempre que falte un dato en esta variable, no faltará el dato asociado a las otras variables.

In [ ]:
msno.heatmap(fifa23, fontsize=12, figsize=(6, 5))
plt.show()

Acá ejemplificamos lo que acabamos de decir.

In [ ]:
faltantes = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'goalkeeping_speed']
fifa23[faltantes].iloc[45:55]

Analizando los datos faltantes, están asociados a ser o no ser arquera. Esto lleva a que podríamos diferenciar el dataset entre arqueras y no arqueras. También esto nos da una pauta de qué características responden a alguien que recorre la cancha respecto a alguien que se queda al arco.

In [ ]:
fifa23[faltantes].info()

print(f"\n\nCantidad de arqueras: {len(fifa23[fifa23['player_positions'] == 'GK'])}")

**Nota:** Además de lo que acabamos de ver, no se cuenta con una descripción precisa para las variables `pace`, `shooting`, `passing`, `dribbling`, `defending` y `physic`. En algunos casos, son superadas por otras variables. Se decide entonces eliminar estas variables.

In [ ]:
fifa23.drop(inplace=True, columns=['pace', 'shooting', 'passing', 'dribbling', 
                                   'defending', 'physic'])

## Simplificación de valores en variables

El `work_rate` de cada jugadora tiene el formato ataque/defensa, catalogando cada uno en `Low`, `Medium` y `High`. Desglosamos esta variable en 2 variables diferentes: `att_work_rate` y `def_work_rate`. Luego, eliminamos la variable original.

In [ ]:
fifa23['att_work_rate'] = [i.split('/')[0] for i in fifa23['work_rate']]
fifa23['def_work_rate'] = [i.split('/')[1] for i in fifa23['work_rate']]
fifa23.drop(columns=['work_rate'],inplace=True)

Los puntajes de las posiciones aparecen con sumas y restas. Queremos que sean sólo enteros.

In [ ]:
for col in ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
            'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 
            'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']:
    new = []
    for i in fifa23[col]:
        if len(i.split('+')) == 2:
            new.append(int(i.split('+')[0])+int(i.split('+')[1]))
        elif len(i.split('-')) == 2:
            new.append(int(i.split('-')[0])-int(i.split('-')[1]))
        else:
            new.append(int(i))

    fifa23[col] = new

## Simplificación de variables

Vamos a simplificar el nombre de varias variables.

In [ ]:
varsdic = {'short_name' : 'name', 'player_positions' : 'position', 
           'height_cm' : 'height', 'nationality_name' : 'nationality', 
           'attacking_crossing' : 'crossing', 
           'attacking_finishing' : 'finishing', 
           'attacking_heading_accuracy' : 'heading', 
           'attacking_short_passing' : 'short_passing', 
           'attacking_volleys' : 'volleys', 'skill_dribbling' : 'dribbling', 
           'skill_curve' : 'curve', 'skill_fk_accuracy' : 'fk_acc', 
           'skill_long_passing' : 'long_passing', 
           'skill_ball_control' : 'ball_control', 
           'movement_acceleration' : 'acceleration', 
           'movement_sprint_speed' : 'sprint', 'movement_agility' : 'agility', 
           'movement_reactions' : 'reactions', 'movement_balance' : 'balance', 
           'power_shot_power' : 'shot_power', 'power_jumping' : 'jumping', 
           'power_stamina' : 'stamina', 'power_strength' : 'strength', 
           'power_long_shots' : 'long_shots', 
           'mentality_aggression' : 'aggression', 
           'mentality_interceptions' : 'interceptions', 
           'mentality_positioning' : 'positioning', 
           'mentality_vision' : 'vision', 'mentality_penalties' : 'penalties', 
           'mentality_composure' : 'composure', 
           'defending_marking_awareness' : 'marking', 
           'defending_standing_tackle' : 'standing_tackle', 
           'defending_sliding_tackle' : 'sliding_tackle', 
           'goalkeeping_diving' : 'diving', 'goalkeeping_handling' : 'handling', 
           'goalkeeping_kicking' : 'kicking_gk', 
           'goalkeeping_positioning' : 'positioning_gk', 
           'goalkeeping_reflexes' : 'reflexes', 
           'goalkeeping_speed' : 'speed_gk'}

In [ ]:
fifa23.rename(columns = varsdic, inplace = True)

In [ ]:
# Se reordenan las columnas
fifa23 = fifa23[['name', 'nationality', 'age', 'height', 'position', 
                 'preferred_foot', 'weak_foot', 'att_work_rate', 
                 'def_work_rate', 'overall', 'potential', 'crossing', 
                 'finishing', 'heading', 'short_passing', 'volleys', 'marking', 
                 'standing_tackle', 'sliding_tackle', 'acceleration', 'sprint', 
                 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 
                 'stamina', 'strength', 'long_shots', 'dribbling', 'curve', 
                 'fk_acc', 'long_passing', 'ball_control', 'aggression', 
                 'interceptions', 'positioning', 'vision', 'penalties', 
                 'composure', 'diving', 'handling', 'kicking_gk', 
                 'positioning_gk', 'reflexes', 'speed_gk', 'ls', 'st', 'rs', 
                 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 
                 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 
                 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']]

fifa23

## Guardado del dataset

El dataset que guardamos consta de 7425 registros con 73 variables.

In [ ]:
fifa23.to_csv('fifa2023.csv', index=False)

## Diccionario

Diccionario del dataset generado hasta ahora.

> Identificación de la jugadora
* `name`: nombre de la jugadora. Se usará sólo para identificar los resultados.
* `nationality`: nacionalidad de la jugadora. Se usará sólo para identificar los resultados.
* `age`: edad de la jugadora.
* `height`: altura de la jugadora, en centímetros.

> Características generales
* `position`: posición en la cancha de la jugadora, denotada con siglas. Puede tener más de una. Cada una de estas opciones representa a su vez una variable que determina el puntaje asignado en dicha posición, del 1 al 99. Las posibilidades son:
    * `ls`: pateadora izquierda.
    * `st`: pateadora central.
    * `rs`: pateadora derecha.
    * `lw`: volante delantera izquierda.
    * `lf`: delantera izquierda.
    * `cf`: delantera central.
    * `rf`: delantera derecha.
    * `rw`: volante delantera derecha.
    * `lam`: mediocampista ofensiva izquierda.
    * `cam`: mediocampista ofensiva central.
    * `ram`: mediocampista ofensiva derecha.
    * `lm`: mediocampista izquierda.
    * `lcm`: mediocampista centro-izquierda.
    * `cm`: mediocampista central.
    * `rcm`: mediocampista centro-derecha.
    * `rm`: mediocampista derecha.
    * `lwb`: volante defensora izquierda.
    * `ldm`: mediocampista defensiva izquierda.
    * `cdm`: mediocampista defensiva central.
    * `rdm`: mediocampista defensiva derecha.
    * `rwb`: volante defensora izquierda.
    * `lb`: defensora izquierda.
    * `lcb`: defensora centro-izquierda.
    * `cb`: defensora central.
    * `rcb`: defensora centro-derecha.
    * `rb`: defensora derecha.
    * `gk`: arquera.
* `preferred_foot`: pie más hábil (izquierdo o derecho).
* `weak_foot`: qué tan bien juega con su pie menos hábil, puntuado del 1 al 5.
* `att_work_rate`: qué tanto se esfuerza la jugadora en términos ofensivos. Se evalua como low, medium y high.
* `def_work_rate`: qué tanto se esfuerza la jugadora en términos defensivos. Se evalua como low, medium y high.
* `overall`: desempeño general y valoración de la jugadora, basado en las habilidades por posición y en su reputación internacional. Valores desde 1 a 99.
* `potential`: calificación general máxima que se espera que alcance la jugadora en la cima de su carrera. Valores desde 1 a 99.

> Características ofensivas
* `crossing`: habilidad para hacer un pase largo desde los lados hacia el centro. Valores desde 1 a 99.
* `finishing`: habilidad para concretar un gol. Valores desde 1 a 99.
* `heading`: habilidad para dar pases o hacer goles, usando la cabeza. Valores desde 1 a 99.
* `short_passing`: exactitud con que la jugadora da pases cortos. Valores desde 1 a 99.
* `volleys`: habilidad para hacer voleas. Valores desde 1 a 99.

> Características defensivas
* `marking`: habilidad de aplicar estrategias que prevengan que el equipo contrario tome la pelota. Valores desde 1 a 99.
* `standing_tackle`: capacidad de quitarle la pelota a la rival, manteniéndose de pie. Valores desde 1 a 99.
* `sliding_tackle`: capacidad de quitarle la pelota a la rival, deslizándose por el suelo. Valores desde 1 a 99.

> Movimientos
* `acceleration`: qué tan rápido alcanza su máxima velocidad. Valores desde 1 a 99.
* `sprint`: velocidad de pique. Valores desde 1 a 99.
* `agility`: destreza de la jugadora a la hora de moverse, teniendo el control de la pelota. Valores desde 1 a 99.
* `reactions`: velocidad de reacción de la jugadora a su entorno. Valores desde 1 a 99.
* `balance`: habilidad de la jugadora para mantenerse estable mientras corre, lleva y controla la pelota. Valores desde 1 a 99.

> Potencia
* `shot_power`: potencia de la jugadora al patear la pelota. Valores desde 1 a 99.
* `jumping`: potencia de la jugadora al saltar. Valores desde 1 a 99.
* `stamina`: resistencia de la jugadora (durabilidad en un mismo partido). Valores desde 1 a 99.
* `strength`: fuerza física de la jugadora. Valores desde 1 a 99.
* `long_shots`: exactitud de la jugadora al hacer pases largos. Valores desde 1 a 99.

> Habilidades físicas
* `dribbling`: habilidad para desplazarse sin perder la pelota. Valores desde 1 a 99.
* `curve`: habilidad de patear una pelota curva al pasar o al apuntar al arco. Valores desde 1 a 99.
* `fk_acc`: habilidad para hacer gol desde un tiro libre. Valores desde 1 a 99.
* `long_passing`: habilidad para hacer pases largos. Valores desde 1 a 99.
* `ball_control`: habilidad para controlar la pelota. Valores desde 1 a 99.

> Otras destrezas
* `aggression`: nivel de agresividad de la jugadora a la hora de empujar, tirar o quitar la pelota. Valores desde 1 a 99.
* `interceptions`: habilidad de la jugadora para interceptar la pelota cuando se la pasan entre las del equipo contrario. Valores desde 1 a 99.
* `positioning`: habilidad para situarse en la posición adecuada al momento de recibir la pelota o anotar goles. Valores desde 1 a 99.
* `vision`: conciencia de la jugadora respecto a las demás jugadoras de su equipo para hacer jugadas en conjunto. Valores desde 1 a 99.
* `penalties`: habilidad para hacer gol en un penal. Valores desde 1 a 99.
* `composure`: habilidad de la jugadora para mantener la calma y manejar la frustración dentro durante el partido. Valores desde 1 a 99.

> Habilidades como arquera
* `diving`: habilidad de la arquera para tirarse de palomita. Valores desde 1 a 99.
* `handling`: habildiad de la arquera para manipular la pelota (con las manos). Valores desde 1 a 99.
* `kicking_gk`: habildiad de la arquera para manipular la pelota (con los pies). Valores desde 1 a 99.
* `positioning_gk`: habilidad de la arquera para situarse en la posición adecuada al momento de recibir la pelota. Valores desde 1 a 99.
* `reflexes`: velocidad en los reflejos de la arquera. Valores desde 1 a 99.
* `speed_gk`: velocidad en los movimientos de la arquera. Valores desde 1 a 99.

---
# Exploración y visualización

In [ ]:
if 'fifa23' not in locals():
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import missingno as msno

    pd.set_option('display.max_columns', 150)
    pd.set_option('display.max_rows',150)
    sns.set_context('talk')
    sns.set_theme(style='white')
    
    fifa23 = pd.read_csv('fifa2023.csv')

fifa23

## Delanteras

* `ls`: pateadora izquierda.
    * `st`: pateadora central.
    * `rs`: pateadora derecha.
    * `lw`: volante delantera izquierda.
    * `lf`: delantera izquierda.
    * `cf`: delantera central.
    * `rf`: delantera derecha.
    * `rw`: volante delantera derecha.
    * `lam`: mediocampista ofensiva izquierda.
    * `cam`: mediocampista ofensiva central.
    * `ram`: mediocampista ofensiva derecha.
    * `lm`: mediocampista izquierda.
    * `lcm`: mediocampista centro-izquierda.
    * `cm`: mediocampista central.
    * `rcm`: mediocampista centro-derecha.
    * `rm`: mediocampista derecha.
    * `lwb`: volante defensora izquierda.
    * `ldm`: mediocampista defensiva izquierda.
    * `cdm`: mediocampista defensiva central.
    * `rdm`: mediocampista defensiva derecha.
    * `rwb`: volante defensora izquierda.
    * `lb`: defensora izquierda.
    * `lcb`: defensora centro-izquierda.
    * `cb`: defensora central.
    * `rcb`: defensora centro-derecha.
    * `rb`: defensora derecha.
    * `gk`: arquera.

In [ ]:
delanteras = fifa23[fifa23['position'] == 'CDM']
delanteras

## Mediocampistas

## Defensoras

## Arqueras